# Preliminaries

## Re-generate the token table

The cell below uses Jupyter's "magic" commands to execute the whole first notebook and create `token_table.csv`.
- In the first line, we use `%%capture` to suppress all the messages that the first notebook normally would print to the screen. 
- In the second line, we use `%run` to run the whole notebook from start to finish.

This takes a little time. When it's done, the CSV file should appear.

In [ ]:
%%capture
%run "03 - VF vs Seneca - Part 1.ipynb"

## Import statements

As usual, we need to import some functionality that isn't part of base Python. [**Pandas**](https://pandas.pydata.org/) and [**Matplotlib**](https://matplotlib.org/stable/index.html) are probably familiar by now. The third package, [**scikit-learn**](https://scikit-learn.org/stable/) is a machine-learning package that provides some very useful methods for dimensionality reduction and classification.

In [ ]:
# provides functionality for tabular data
import pandas as pd

# provides plotting
from matplotlib import pyplot as plt

# dimensionality reduction
from sklearn.decomposition import PCA

## Load the data

This should load the token table from the CSV file and display it on the screen. **Note** I'm adding an extra step to force all the values to be read as text. This is because otherwise the **person** column tends to be read as numbers.

In [ ]:
# load the token table
token_table = pd.read_csv('token_table.csv', dtype=str)

# show the results
display(token_table)

# Feature extraction

## Cleaning the data

As we did last time, here we create a new table leaving out any tokens labelled as punctuation.

In [ ]:
no_punct = token_table.loc[token_table.upos!='PUNCT']

## Calculating lemma frequencies

### Feature tallies

Here we use the Pandas method `crosstab()` to create a feature frequency table. If we cross-tabulate **author** and **title**, on the one hand, against **lemma** on the other, then the resulting table has one row for each **author** - **title** pair, and one column for each possible value in **lemma**. The values tell us how many times each lemma occurs in each text.

### Normalization

As we did last time, we're going to normalize our counts by dividing by the length of each text. That gives us frequencies instead of raw tallies, making the texts more easily comparable.

Because the resulting frequencies are often very low, we're going to follow standard practice in stylometry and multiple them all by 1000. That means that values represent the expected count for a given feature in a sample of 1000 words.

The code below produces a table with one column for every possible lemma.

In [ ]:
# tally lemma counts and normalize
lemma_freq = pd.crosstab([no_punct.author, no_punct.title], no_punct.lemma, normalize='index')

# multiply by 1000
lemma_freq = lemma_freq * 1000

# display the result
display(lemma_freq)

But as we learned last time, many of these columns are unhelpful to us because they're dedicated to lemmata that occur very rarely in the corpus. Let's see if we can do some more precise shaping of this feature set based on overall frequency in the corpus.

### Overall counts

Let's calculate again the corpus-wide count for each lemma. These will allow us to order the lemmata by frequency and identify the **most frequent words** as well as **hapax legomena**.

In [ ]:
# calculate corpus-wide counts for all lemmata
lemma_count = no_punct.lemma.value_counts()

### MFWs

What are the most frequent words in the corpus?

In [ ]:
# select the top 25 lemmata
mwf = pd.DataFrame(lemma_count[:25])

display(mwf)

### Hapax legomena

As we saw last time, over half of the lemmata in this list have a count of 1: they are **hapax legomena** (i.e. ἅπαξ λεγόμενα, "said once").

In [ ]:
# select only those lemmata that occur once
hapaxes = pd.DataFrame(lemma_count[lemma_count==1])

display(hapaxes)

### Stoplist

A **stoplist** is a list of features we will exclude from our analysis. For this notebook, I'm somewhat arbitrarily going to create a lemma stoplist including all lemmata that occur fewer than 10 times in the corpus.

In [ ]:
# identify the lemmata for which count is less than 10
lemma_stop = lemma_count.index.values[lemma_count<10]

# how many?
print(len(lemma_stop), 'lemmata to be excluded')

### Revised lemma table

Let's redo the lemma table. After we calculate the frequencies, we drop all the columns that are in the stoplist. The data frame `drop()` method lets us drop rows or columns, so we specify that we're talking about columns here.

In [ ]:
# tally lemma counts and normalize
lemma_freq = pd.crosstab([no_punct.author, no_punct.title], no_punct.lemma, normalize='index')

# multiply by 1000
lemma_freq = lemma_freq * 1000

# reorder columns according to overall frequency
lemma_freq = lemma_freq[lemma_count.index.values]

# drop columns in the stoplist
lemma_freq = lemma_freq.drop(lemma_stop, axis='columns')

# display the result
display(lemma_freq)

Much better! We only have 1400 features to deal with here, a 90% reduction from the initial table.

### Visualizing feature distributions

As we saw last time, we can use a [violin plot](https://en.wikipedia.org/wiki/Violin_plot) to visualize the distribution of a given feature across the corpus. This can be helpful when comparing classes, say, comparing the 8 books by Valerius Flaccus to the 8 plays by Seneca.

This is the code that we finished with last time. In the first lines, we select a feature and assign it to the variable `feat`: this must be the name of one of the columns in our frequency table. The remainder of the code draws and labels a graph comparing the distribution of that feature across the books of each author.

In [ ]:
# feature of interest
feat = 'amor'

# create empty list for the labels and data values
labels = []
values = []

# extract labels, data from grouped table
for label, group in lemma_freq.groupby('author'):
    labels.append(label)
    values.append(group[feat])

# create a new plot
fig, ax = plt.subplots()

# plot the values
ax.violinplot(values)

# add labels
ax.set_xticks(range(1, len(labels)+1), labels=labels)
ax.set_ylabel('occurrance per 1000 words')
ax.set_title(feat)

# show
plt.show()

<div class="alert alert-success">
<h3>🤔 Explore the data</h3>

<p style="margin:1em 0em">Before continuing, try replacing <code>amor</code> above with various other lemmata of your choice. You can pick words that seem interesting to you, work through the MFWs listed above, or maybe try some of the words from Bernhard Söllradl's paper.</p>

<p style="margin:1em 0em">If you enter a word that doesn't have a column in the table, you'll get an error. Just pick another word and try again. Note that Patrick's trained language model tends to prefer 'u' over 'v', so for example <code>vir</code> doesn't work but <code>uir</code> does.</p>
    
<p style="margin:1em 0em"><strong>Which words show large differences between the two authors?</strong></p>
</div>

<hr>

# Basic clustering and classification

The next step in our experiment is to try **clustering** the data. This is a common task in stylometry and in data science generally. In the simplest terms, we're trying to spread out all the samples as widely as we can, and see whether they fall into distinct groups, or "clusters".

A related task is **classification**: deciding which group a given sample belongs to. If we have a mathematical model that defines the clusters, we can then use that model to classify an unknown sample.

For example, take a look at the distributions of the feature **que** using the code block above. Now, imagine we come across a text whose authorship is disputed. It uses *-que* at a rate of 22 times per 1000 words. Which of the two authors do you think is most likely to have written it?

## Visualizing multiple features at once

So far we've looked at one feature at a time. But we also know that several, perhaps many, of the lemmata differ between the two authors. Moreover, combining features may bring out information that we don't see otherwise.

Let's start by visualizing just two lemma frequencies at once... say **et** and **que**. We can plot these in a Cartesian space—that is, an x-y plot—with the frequency of **et** as the horizontal axis and that of **que** as the vertical. Each text then becomes a point on this graph, positioned according to its use of the two features.

The code below works much the same as the previous block: the two features of interest are set at the top; the remainder of the cell creates the graph. Drawing a **scatter plot** or x-y graph is really straightforward in Pyplot. Can you see more or less what's going on in each step?

💁🏻‍♂️ Hint: adding the `label=` option to `scatter()` registers the series with a given name. That's how Pyplot is able to create a legend at the end.

In [ ]:
# features of interest
x_feat = 'et'
y_feat = 'que'

# create a new plot
fig, ax = plt.subplots()

# iterate over the authors
for label, group in lemma_freq.groupby('author'):
    
    # add a series points for this author
    ax.scatter(group[[x_feat]], group[[y_feat]], label=label)

# add figure labels and legend
ax.set_xlabel(x_feat + ' (per 1000 words)')
ax.set_ylabel(y_feat + ' (per 1000 words)')
ax.legend()

# show
plt.show()

<div class="alert alert-success">
<h3>🤔 Explore the data</h3>

<p style="margin:1em 0em">As before, try plugging in lemmata of your choice here to see the resulting graph. Do the two authors' texts separate out, or do they overlap? If you get good separation, see if you can formalize it for yourself: try writing out a set of rules for classifying the texts according to feature frequency.</p>
</div>

<hr>

# Dimensionality reduction

A scatterplot works well for representing exactly two features, but we have 1400 in our table. How can we visualize the data in a way that uses as much of the available information as possible? This is the problem of **dimensionality reduction**: reducing 1400 dimensions to 2, for example, so that we can draw a plot, while minimizing information loss.

## Principal Components Analysis

One common technique is **principal components analysis**, or **PCA**. This involves projecting data into a new *n*-dimensional space... where *n* is a more convenient number than 1400.

The new dimensions, called **principal components**, are essentially a rotation and stretching of the original dimensions. For example, PC1 might be calculated using a formula that multiplies each dimension by some weight, or coefficient. For example,

$$
0.002963 \times \textsf{que} + 0.000504 \times \textsf{et} + 0.002526 \times \textsf{qui} + \dots + w_i \times X_i
$$

where $w_i$ is the *i*th weight and $X_i$ is the *i*th dimension.

This transformation is calculated so that PC1 represents the greatest possible variance among the data points. The second PC is the dimension orthogonal to PC1 that shows the next greatest variance, and so on.

### Performing PCA using sklearn

Actually calculating these weights and then projecting the data into the new space is a complicated operation. A popular tool for this process is the **scikit-learn** package, aka **sklearn**. The cell below instantiates a new 2-dimensional PCA model and then transforms our data.

In [ ]:
# calculate the principal components
pca_model = PCA(n_components=2)

# project the data into the lower-dimensional space 
pca_feats = pca_model.fit_transform(lemma_freq)

# turn the PCA feature set into a data frame
pca_feats = pd.DataFrame(pca_feats, columns=['PC1', 'PC2'])

### Plotting PCs

Now that our 1400 lemmata have been projected down to two dimensions, we can plot them on a scatter plot. Because the new 2-dimensional dataset is in a different table, we need to call back to the original table for the author labels.

The new dimensions do not have meaningful units; rather, the relative positions of and distance between the points are what we're looking at.

In [ ]:
# create a new plot
fig, ax = plt.subplots()

# iterate over the authors
for label, group in pca_feats.groupby(lemma_freq.index.get_level_values('author')):
    
    # add a series points for this author
    ax.scatter(group.PC1, group.PC2, label=label)

# add figure labels and legend
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.legend()

# show
plt.show()

# Integrating additional features

Besides lemmata, Bernhard also uses part-of-speech features like interjections, and grammatical features like mood and voice. We have these data in additional columns in our original token table. Here we extract those additional features and then put everything into a single feature frequency table.

## Part of speech

We'll extract part of speech frequencies, initially as a new table. We can combine the tables at the end. We use `crosstab()` here just as we did for lemmata above.

In [ ]:
# tally pos counts and normalize
pos_freq = pd.crosstab([no_punct.author, no_punct.title], no_punct.upos, normalize='index')

# multiply by 1000
pos_freq = pos_freq * 1000

# display the result
display(pos_freq)

### Renaming columns

Something we might want to watch out for if we ultimately add all these tables together is possible overlap between column names. Off the top of my head, I can't imagine a specific example, but I want to make sure we don't end up with a column in the **mood** table that has the same name as a column in the **lemma** table. 

The easiest way to make sure that all the column labels are unique is to add a prefix derived from the table name. In the code below, I redo the frequency table, with an extra step to rename all the columns so that they start with `pos_`. 

In [ ]:
# tally pos counts and normalize
pos_freq = pd.crosstab([no_punct.author, no_punct.title], no_punct.upos, normalize='index')

# multiply by 1000
pos_freq = pos_freq * 1000

# rename columns with a prefix
pos_freq = pos_freq.rename(columns = lambda name: 'pos_' + name)

# display result
display(pos_freq)

## Morphological features

If you remember our last couple of notebooks, morphological features that we pulled from SpaCy's `morph` attribute are spread across several columns in the token table: **mood**, **voice**, **tense**, **person**, **number**, **gender**, and **case**. I could use the same technique to create an additional feature frequency table for each of these, but that's a lot of tables to manage individually.

Instead, working from the model of the part-of-speech table, I'm going to create a template that I can re-use in a **for loop** to generate all the tables at once.

In [ ]:
# a list of columns to process
feature_names = ['mood', 'voice', 'tense', 'person', 'number', 'gender', 'case']

# an empty list to gather the resulting tables
morph_tables = []

# iterate over the columns, using `feat` as a stand-in for the current feature
for feat in feature_names:
    
    # tally feature counts and normalize
    this_freq = pd.crosstab([no_punct.author, no_punct.title], no_punct[feat], normalize='index')

    # multiply by 1000
    this_freq = this_freq * 1000

    # rename columns with a prefix
    this_freq = this_freq.rename(columns = lambda name: feat + '_' + name.upper())
    
    # add table to the list
    morph_tables.append(this_freq)
    
# join all the tables together
morph_freq = pd.concat(morph_tables, axis=1)
    
# show results
display(morph_freq)

## Combining all the features

Now let's combine lemma, part-of-speech, and morphological features in a single feature table and re-do PCA. This will give us the most complete picture of the differences among our texts.

(Note that it is possible that some of the features are muddying the waters—more isn't always better. To get the very best separation, we would have to more carefully examine what each feature is contributing to the PCA and possibly eliminate some that are unhelpful.)

In [ ]:
# combine all the tables
feat_freq = pd.concat([lemma_freq, pos_freq, morph_freq], axis=1)

# re-initialize the PCA model
pca_model = PCA(n_components=2)
pca_feats = pd.DataFrame(
    data = pca_model.fit_transform(feat_freq), 
    columns = ['PC1', 'PC2'], 
    index = feat_freq.index,
)

# create a new plot
fig, ax = plt.subplots()

# iterate over the authors
for label, group in pca_feats.groupby('author'):
    
    # add a series points for this author
    ax.scatter(group.PC1, group.PC2, label=label)

# add figure labels and legend
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.legend()

# show
plt.show()

# Interpretation

## First impressions

Okay, we've thrown everything at the wall here... does anything seem to have stuck? What first impressions do we have of the results? What new questions are raised?

### Separation by author

For one thing, we've shown that authorship attribution with lexical features continues to be a pretty straightforward process. We can separate these 16 texts such that there is no overlap between the authors.

### Internal variance

At the same time, there is at least as much variation *within* the clusters as between them: the distance between the two most widely-separated orange dots is greater than the gap between the orange dots and the blue ones.

In general, the Valerius dots seem to be spread over a wider area than the Seneca dots. Does that make sense, given that Valerius' texts are books of a single epic, whereas Seneca's texts are standalone plays? 

### Outliers

In particular, one of the plays of Seneca is really far from everything else. What's going on there?

## From visualization back to close reading

One thing we can do is label the graph. Knowing the identity of the dots might help guide us in looking back at the text for some answers.

In [ ]:
# create a new plot
fig, ax = plt.subplots()

# iterate over the authors
for label, group in pca_feats.groupby('author'):
    
    # add a series points for this author
    ax.scatter(group.PC1, group.PC2, label=label)

# add figure labels
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

# add annotations for each dot
for row in pca_feats.itertuples():
    ax.text(row.PC1, row.PC2, row.Index[1])

# show
plt.show()

<div class="alert alert-success">
<h3>🤔 Explore the data</h3>

<p style="margin:1em 0em">If you have time, do a little research on Wikipedia, and maybe skim through the texts. Can you brainstorm any hypotheses about why, say, <em>Argonautica</em> 8 and <em>Phoenissae</em> might be further from their respective clusters? Any other observations?</p>
    
<p>For my part, I suddenly realize that I've left out Seneca's <em>Agamemnon</em>, entirely accidentally. I left out <em>Octavia</em> on purpose just to focus on Greek subjects for this exercise, but I somehow just missed <em>Agamemnon</em>. I don't have time to fix it now, but I'll add it back into the first notebook when I can.</p>
</div>